# Setup and test algebraic eqn solver

In [3]:
import sys
import os
import numpy as np
import pandas as pd
import os
from scipy.interpolate import interp1d
from scipy import constants

import matplotlib.pyplot as plt
os.chdir(sys.path[0] + '\..') # change our working directory to the root of the pybamm folder
import pybamm
print(pybamm.__path__[0])
%matplotlib widget

c:\Users\Vivian\Dropbox (University of Michigan)\from_box\Research\PyBaMM\PyBaMM\pybamm


In [4]:

class PouchVenting(pybamm.BaseSubModel):
    """Pouch cell venting accounting for CO2 gas generation from SEI decomposition 
    and electrolyte saturation pressure.

    Parameters
    ----------
    param : parameter class
        The parameters to use for this submodel
    reactions : dict, optional
        Dictionary of reaction terms

    **Extends:** :class:`pybamm.BaseSubModel`
    """

    def __init__(self, param):
        super().__init__(param)

    def get_fundamental_variables(self):
        param = self.param
        delta_sigma = pybamm.Variable("Cell expansion stress [kPa]")
        T_vol_av = pybamm.standard_variables.T_vol_av
        # T_x_av = pybamm.PrimaryBroadcast(T_vol_av, ["current collector"])
        # T_n = pybamm.PrimaryBroadcast(T_x_av, "negative electrode")
        variables = {
            "Cell expansion stress [kPa]": delta_sigma,
            "Volume-averaged cell temperature": T_vol_av,
            # 'Volume-averaged cell temperature [K]': param.therm.Delta_T * T_vol_av
            # + param.T_ref,
        }
        return variables

    def get_coupled_variables(self, variables):
        print(1)

        param = self.param
        m_an = param.therm.m_an
        x_sei_0 = param.therm.x_sei_0
        M_C6 = param.therm.M_an
        V_head_0 = param.therm.V_head_0
        A_surf = param.therm.A_surf
        L = param.therm.L_poron
        E = param.therm.E_poron
        alpha_cell = param.therm.alpha_cell
        P_crit = param.therm.P_crit
        sigma_0 = param.therm.sigma_0
        P_atm = param.therm.P_atm

        x_sei = param.therm.x_sei_0 #variables["Fraction of Li in SEI"]
        delta_sigma = variables["Cell expansion stress [kPa]"]
        T = variables['Volume-averaged cell temperature']
        T_dimensional = param.therm.Delta_T * T + param.T_ref
        delta_T = param.therm.Delta_T * T
        print(2)

        delta_d = L/E*delta_sigma
        V_head = V_head_0 + A_surf*(delta_d-alpha_cell*delta_T)   
        P_sat = param.therm.P_sat(T) #dimensionless
        n_CO2 = m_an*(x_sei_0 - x_sei)/(2*M_C6)
        P_CO2 = (n_CO2*constants.R*T/V_head/1000)/P_crit #kPa
        P_total = P_CO2 + P_sat 

        delta_sigma_gas = P_total-sigma_0-P_atm
        delta_sigma_themal_expansion = E*alpha_cell*delta_T/L
        print(3)

        Q_ohm = pybamm.Scalar(15)
        Q_scale = param.i_typ * param.potential_scale / param.L_x
        variables.update({
            "Electrolyte gas saturation pressure": P_sat/P_crit,
            "CO2 gas pressure": P_CO2/P_crit,
            "Total gas pressure [kPa]": P_total/P_crit,
            "Electrolyte gas saturation pressure [kPa]": P_sat,
            "CO2 gas pressure [kPa]": P_CO2,
            "Total gas pressure [kPa]": P_total,
            "Headspace volume [m3]": V_head,
            "Volume-averaged total heating": Q_ohm,
        })

        return variables


    def set_rhs(self, variables):
        print(4)

        param = self.param
        T_vol_av = variables['Volume-averaged cell temperature']
        Q_vol_av = variables["Volume-averaged total heating"]
        T_amb = param.T_amb(pybamm.t * param.timescale)
        cell_surface_area = param.a_cooling
        cell_volume =  param.v_cell
        total_cooling_coefficient = (
            - param.h_total
            * cell_surface_area
            / cell_volume
            / (param.delta ** 2)
                )
        print(5)

        self.rhs = {
            T_vol_av: (
                 param.B * Q_vol_av + total_cooling_coefficient * (T_vol_av - T_amb)
            )
            / ( param.C_th * param.rho(T_vol_av)),
        }

    def set_algebraic(self, variables):
        print(6)

        param = self.param
        P_crit = param.therm.P_crit
        L = param.therm.L_poron
        E = param.therm.E_poron
        alpha_cell = param.therm.alpha_cell
        P_atm = param.therm.P_atm
        sigma_0 = param.therm.sigma_0
        print(7)

        T = variables['Volume-averaged cell temperature']
        delta_T = param.therm.Delta_T * T
        delta_sigma = variables["Cell expansion stress [kPa]"]
        P_total = variables["Total gas pressure [kPa]"]        
        self.algebraic = {
            delta_sigma: delta_sigma - pybamm.maximum(P_total-sigma_0-P_atm, E*alpha_cell*delta_T/L)
            } #pybamm.maximum(P_total-sigma_0-P_atm, E*alpha_cell*delta_T/L)

    def set_initial_conditions(self, variables):
        # param = self.param
        # sigma_0 = param.therm.sigma_0
        delta_sigma = variables["Cell expansion stress [kPa]"]
        T_vol_av = variables["Volume-averaged cell temperature"]
        self.initial_conditions = {
            delta_sigma: pybamm.Scalar(0), 
            T_vol_av: self.param.T_init,

        }

In [5]:
chemistry = pybamm.parameter_sets.Tran2023
parameters = pybamm.ParameterValues(chemistry = chemistry)
param = pybamm.LithiumIonParameters()

In [6]:
t_end = 300
t_eval = np.linspace(0,t_end,t_end*10)

model = pybamm.BaseModel()

model.submodels = {
    "venting": PouchVenting(param),
    # 'thermal': pybamm.thermal.Lumped(parameters),
}
model.build_model()
display(list(model.variables))
solver = pybamm.CasadiSolver()
# solution = solver.solve(model, t_eval, external_variables={'X-averaged negative electrode temperature':50+273.15})
solution = solver.solve(model, t_eval)

1
2
3
4
5
6
7


['Cell expansion stress [kPa]',
 'Volume-averaged cell temperature',
 'Electrolyte gas saturation pressure',
 'CO2 gas pressure',
 'Total gas pressure [kPa]',
 'Electrolyte gas saturation pressure [kPa]',
 'CO2 gas pressure [kPa]',
 'Headspace volume [m3]',
 'Volume-averaged total heating']

NotImplementedError: method self.evaluate() not implemented for symbol Typical current [A] of type <class 'pybamm.expression_tree.parameter.Parameter'>

In [3]:
model = pybamm.BaseModel()
model.submodels = {"venting": PouchVenting(None, "Negative")}
model.build_model()

solver = pybamm.CasadiSolver()
sim = pybamm.Simulation(
        model,
        solver=solver,
)
sim.solve([0,1])  
sim.plot(['u','v'])

interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…